In [ ]:
import requests

NUM_REPS = 30
API_URL = 'http://localhost/api/des'

In [ ]:
response = requests.get(f'{API_URL}/jobs')
response.json()

In [ ]:
with open('../../assets/config_base.xlsx', 'rb') as fp:
    response = requests.post(
        f'{API_URL}/jobs',
        files={'config_bytes': ('config_base.xlsx', fp.read())},
        data={
            'sim_hours': 168*10.0,
            'num_reps': 30,
            'runner_speed': 1.2
        },
        timeout=30
    )

assert response.status_code == 202
_id = response.json()['id']

In [ ]:
from time import sleep
from IPython.display import clear_output, display

while True:
    clear_output(wait=True)
    response = requests.get(f'{API_URL}/jobs/{_id}/status')
    assert response.status_code == 200
    display(status := response.json())
    if status['progress'] == status['max_progress']:
        break
    sleep(5)

In [ ]:
response = requests.get(f'{API_URL}/jobs/{_id}/results/0')
assert response.status_code == 200

In [ ]:
response.json().keys()

In [ ]:
data = [None]*NUM_REPS

for idx in range(NUM_REPS):
    response = requests.get(f'{API_URL}/jobs/{_id}/results/{idx}')
    assert response.status_code == 200
    data[idx] = response.json()

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import sem, t as student_t

import plotly.express as px
import plotly.graph_objects as go

### Utilisation

In [ ]:
def timeseries_mean(df: pd.DataFrame, time_col = 't', val_col = 'x'):
    _tmp = df.copy()
    # diff() starts with 'NaN' in first row, so shift(-1) is needed
    return float(np.average(_tmp[val_col][:-1], weights=_tmp[time_col].diff().shift(-1)[:-1]))

def mean_claimed(data: dict, resource_name: str):
    df = pd.DataFrame(
        data['resources']['n_claimed'][resource_name],
        columns=['t','x']
    )
    return timeseries_mean(df)

def mean_available(data: dict, resource_name: str):
    df = pd.DataFrame(
        data['resources']['capacity'][resource_name],
        columns=['t','x']
    )
    return timeseries_mean(df)

def utilisation(data):
    return {
        res: {
            'median': np.quantile([mean_claimed(dd, res)/mean_available(dd, res) for dd in data], 0.5)
        }
        for res in data[0]['resources']['n_claimed'].keys()
    }

In [ ]:
df = pd.DataFrame.from_dict(utilisation(data), orient='index').sort_values(by='median', ascending=False)
df2 = pd.concat([
    pd.DataFrame([[res, mean_claimed(dd, res)/mean_available(dd, res)] for dd in data],
             columns=['label', 'value'])
    for res in data[0]['resources']['n_claimed'].keys()
])
fig = px.box(df2, 'label', 'value')
fig.update_xaxes(categoryorder='array', categoryarray=df.index)
fig

### WIP

In [ ]:
def wip_df(data, wip):
    df = pd.DataFrame(data['wips'][wip], columns=['t','x']).set_index('t')
    df.index = pd.to_timedelta(df.index, unit='h')
    df = df.resample('h').mean().ffill()
    return df

def wips_df(data, wip):
    df = pd.concat([wip_df(dd, wip) for dd in data], axis=1)
    df2 = pd.DataFrame({
        'q10': df.quantile(0.1, axis=1),
        'q25': df.quantile(0.25, axis=1),
        'median': df.quantile(0.5, axis=1),
        'q75': df.quantile(0.75, axis=1),
        'q90': df.quantile(0.9, axis=1)
    })
    df2.index = df2.index / pd.Timedelta(days=1)
    return df2

In [ ]:
df = wips_df(data, 'Total WIP')

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x = df.index,
        y=df.q90,
        line_width=0,
        line_color='rgba(255,255,255,0)',
    )
)
fig.add_trace(
    go.Scatter(
        x = df.index,
        y=df.q10,
        fill='tonexty',
        fillcolor='rgba(0,0,255,0.2)',
        mode='none',
    )
)
fig.add_trace(
    go.Scatter(
        x = df.index,
        y=df.q75,
        line_width=0,
        line_color='rgba(255,255,255,0)',
    )
)
fig.add_trace(
    go.Scatter(
        x = df.index,
        y=df.q25,
        fill='tonexty',
        fillcolor='rgba(0,0,255,0.4)',
        mode='none',
    )
)
fig.add_trace(
    go.Scatter(
        x = df.index,
        y=df['median'],
        line_color='rgb(0,0,0)'
    )
)

fig.update_layout(showlegend=False, title='Total WIP')
fig.update_xaxes(title='Days')
fig.update_yaxes(title='Work in progress')
fig

### Lab TAT

In [ ]:
from itertools import chain
from math import ceil


lab_tats = [[
    (x['qc_end']-x['reception_start'])/24.0
    for x in data[n]['specimen_data'].values()
    if 'reporting_end' in x
] for n in range(len(data))]
lab_tats = list(chain(*lab_tats))

df = pd.DataFrame(lab_tats, columns=['x'])

fig = px.histogram(df, x='x', nbins=28, histnorm='probability')
fig.update_traces(
    xbins={
        'start': 0,
        'end': ceil(max(df.x)),
        'size': 1
    }
)
fig.update_layout(title='Lab turnaround time')
fig.update_xaxes(title='Days')
fig.update_yaxes(title='Probability')
fig